download data

In [ ]:
!rm -rf data
!wget -P data https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/VKRecsysChallenge2024/dataset/train_interactions.parquet
!wget -P data https://storage.yandexcloud.net/ds-ods/files/files/c1992ccf/users_meta.parquet
!wget -P data https://storage.yandexcloud.net/ds-ods/files/files/13b479ed/items_meta.parquet
!wget -P data https://storage.yandexcloud.net/ds-ods/files/files/0235d298/test_pairs.csv
!wget -P data https://storage.yandexcloud.net/ds-ods/files/files/55b07019/sample_submission.csv

In [ ]:
!wget https://storage.yandexcloud.net/ds-ods/files/files/82d94c5c/baseline.ipynb